In [1]:
import os
import numpy as np
import networkx as nx

import process
import pandas

# Chip analysis

The following notebook is an analysis notebook. It has been conceived to order and analyse a dump of confocal images. All questions concerning the code should be emailed to gustave.ronteix@pasteur.fr.

### Processing

The grunt functions are called from process.

Attention: verify the channel order.

In [2]:
livePosition = 2
green = 0
orange = 1

channels = [livePosition, green, orange]

zRatio = 1/3
rNoyau = 9
dCells = 70

path = r'\\atlas.pasteur.fr\Multicell\Gustave\fusionGustave\TIFF\0'

In [ ]:
process._sortFiles(path)

In [3]:
process._makeSpheroidClass(path, zRatio, rNoyau, dCells, channels)

prep image: 1 folder and time 0


AttributeError: 'list' object has no attribute 'mean'

Remarquons que la segmentation du spheroide dans un ou l'autre des types de cellules depend ici du classificateur naif que nous avons cree. Il repose sur un fit lineaire des points en fonction de leur couleur. On trouve que ce dernier doit avoir comme pente:

$a = \frac{<xy>}{<x^2>}$

Le signe par rapport a celui-ci est donne par:

$sign = y - a*x$

Ce qui nous donne par consequence un moyen rapide pour affecter un genre a chaque cellule.

In [ ]:
IntensityPath = r'X:\Gustave\Experiments\Nuclei Segmentation\04072019\Seb\tif\Spheroids\\'

In [ ]:
df1 = pandas.read_csv(IntensityPath + 'intensityFrame_00_00.csv')
df2 = pandas.read_csv(IntensityPath + 'intensityFrame_01_00.csv')

In [ ]:
import matplotlib.pyplot as plt


ax1 = df1.plot.scatter(x='Orange',y='Green', c= 'r')
df2.plot.scatter(x='Orange',y='Green', ax = ax1)
plt.plot(df['Orange'], a*df['Orange'], 'g-')

In [ ]:
df = df1.append(df2)

In [ ]:
a = df['Orange'].dot(df['Green'])/df['Orange'].dot(df['Orange'])

In [ ]:
len(df.loc[df['Green']-a*df['Orange']<0])

### Refined analysis

We use the refined metrics of networkx to add centrality, degree and other measures to the death/survival rates of the spheroid. The following process follows:

 - open json file
 - modify the spheroid properties
 - save global properties in soheroid file
 - save file

In [ ]:
def _refineSph(dic):
    
    G = _makeG(dic)
    
    A = networkx.betweenness_centrality(G) # betweeness centrality
    B = networkx.clustering(G)
    C = networkx.degree(G)
    
    for v in G:
        
        dic['cells'][v]['degree'] = C[v]
        dic['cells'][v]['clustering'] = B[v]
        dic['cells'][v]['centrality'] = A[v]
        
    dic['N'] = len(dic['cells'])
    dic['assortativity'] = nx.degree_assortativity_coefficient(G)
    dic['average degree'] = np.asarray([float(C[v]) for v in G]).mean()
    
    return dic

In [ ]:
_refineSph(_Sph)

process._saveSpheroid(_Sph.Spheroid, path + '\spheroid_' + spheroidFolder + r'_' +  timeFolder + '.json')